# 2장. &#129303;Transformers 라이브러리 사용하기
- [강좌 링크](https://wikidocs.net/166794)

1장에서 보았듯 Transformer 모델은 규모가 매우 커 이를 해결하기 위해 &#129303;Transformers 라이브러리가 만들어지게 되었고 특징은 다음과 같다.
- 사용 용이성, Ease of use
- 유연성, Flexibility
- 단순성, Simplicity

# 1. Pipeline 내부 실행 과정
1. 전처리(Preprocessing)
2. Model
3. 후처리(Postprocessing)

## Tokenizer를 이용한 Preprocessing

다른 신경망과 마찬가지로 Transformer 모델 또한 원시 텍스트를 직접 처리할 수 없으므로 모델이 이해할 수 있는 숫자로 바꿔주게 됨.
- 입력을 token이라 불리는 { word | subword | symbol }로 분할
- 각 token을 정수로 mapping
- 모델에 유용할 수 있는 additional inputs를 추가

이 모든 preprocess는 모델이 pretraining될 때와 정확히 동일한 방식으로 수행되어야 하므로 Model Hub에서 해당 정보를 다운로드해야 함.

In [1]:
from custom_utils import *

from transformers import AutoTokenizer

wrapper = CustomObject()

# sentiment-analysis 파이프라인의 default checkpoint
wrapper.checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
wrapper.tokenizer = AutoTokenizer.from_pretrained(wrapper.checkpoint)

한 번 Tokenizer를 생성하면 아래 코드처럼 문장을 입력하여 모델에 바로 전달할 수 있는 Python Dictionary 정보를 구할 수 있음.

이후 해야할 일은 input_ids 리스트를 텐서로 변환하는 것뿐임.

In [2]:
wrapper.raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
]
wrapper.inputs = wrapper.tokenizer(wrapper.raw_inputs, padding = True, truncation = True, return_tensors = "pt")
print(wrapper.inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


## Model 살펴보기

Tokenizer와 동일한 방법으로 다운로드 가능.

AutoModel의 from_pretrained() 활용

In [4]:
from transformers import AutoModel

wrapper.model = AutoModel.from_pretrained(wrapper.checkpoint)
print(wrapper.model)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

해당 아키텍처에는 기본 Transformer 모듈만 포함되어 있으며 입력이 주어지면 feature라고도 불리는 hidden_states를 출력한다.

각 모델 입력에 대해 Transformer 모델에 의해 수행된 해당 입력의 문맥적 이해 결과를 나타내는 고차원 벡처를 가져온다

뭔 개소린지 모르겠는데 뒤에서 설명한다니 한 번만 참는다.

## 고차원 벡터?

Transformer 모듈의 벡터 출력은 일반적으로 그 규모가 크다. 일반적으로 3 가지 차원이 있다.
- Batch Size: 한 번에 처리되는 시퀀스의 개수(2)
- Sequence Length: 시퀀스 숫자 표현의 길이(16)
- Hidden Size: 각 모델 입력의 벡터 차원(작은 모델의 경우 768, 큰 모델의 경우 3072 이상일 수도 있음)

사전 처리한 입력을 모델에 넘기면 다음과 같은 내용을 볼 수 있음.

In [5]:
wrapper.outputs = wrapper.model(**wrapper.inputs)
print(wrapper.outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [6]:
print(wrapper.outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1798,  0.2333,  0.6321,  ..., -0.3017,  0.5008,  0.1481],
         [ 0.2758,  0.6497,  0.3200,  ..., -0.0760,  0.5136,  0.1329],
         [ 0.9046,  0.0985,  0.2950,  ...,  0.3352, -0.1407, -0.6464],
         ...,
         [ 0.1466,  0.5661,  0.3235,  ..., -0.3376,  0.5100, -0.0561],
         [ 0.7500,  0.0487,  0.1738,  ...,  0.4684,  0.0030, -0.6084],
         [ 0.0519,  0.3729,  0.5223,  ...,  0.3584,  0.6500, -0.3883]],

        [[-0.2937,  0.7283, -0.1497,  ..., -0.1187, -1.0227, -0.0422],
         [-0.2206,  0.9384, -0.0951,  ..., -0.3643, -0.6605,  0.2407],
         [-0.1536,  0.8988, -0.0728,  ..., -0.2189, -0.8528,  0.0710],
         ...,
         [-0.3017,  0.9002, -0.0200,  ..., -0.1082, -0.8412, -0.0861],
         [-0.3338,  0.9674, -0.0729,  ..., -0.1952, -0.8181, -0.0634],
         [-0.3454,  0.8824, -0.0426,  ..., -0.0993, -0.8329, -0.1065]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


&#129303;Transformers 모델의 출력은 `namedtuple` 또는 dictionary처럼 동작하여 요소에 접근하기 위해 속성 또는 키를 사용할 수 있다.

## Model Heads: 숫자 이해하기

model head는 hidden states의 고차원 벡터를 입력으로 받아 다른 차원에 투영한다.

일반적으로 헤드는 하나 또는 몇 개의 선형 레이어로 구성된다.

Transformer 모델의 출력은 처리할 model head로 직접 전달된다.

&#129303;Transformers에는 다양한 아키텍처가 있으며 각 아키텍처는 특화된 작업을 처리하도록 설계되었다.
- *Model (hidden states를 리턴)
- *ForCausalLM
- *ForMaskedLM
- *ForMultipleChoice
- *ForQuestionAnswering
- *ForSequenceClassification
- *ForTokenClassification
- Others...&#129303;

이 섹션에서의 예시에서는 시퀀스 분류 헤드가 포함되어 있는 모델이 필요하므로 AutoModel 대신 `AutoModelForSequenceClassification`을 사용한다.

In [7]:
from transformers import AutoModelForSequenceClassification

wrapper.model = AutoModelForSequenceClassification.from_pretrained(wrapper.checkpoint)
wrapper.outputs = wrapper.model(**wrapper.inputs)
print(wrapper.outputs.logits.shape)

torch.Size([2, 2])


두 개의 레이블만 있는 모델을 통과한 2개의 모델이므로 결과의 모양은 (2, 2)

## 출력 후처리하기

모델에서 출력으로 얻은 값은 그 자체로 의미있는 값은 아니다.

In [8]:
print(wrapper.outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


이는 모델의 마지막 계층에서 출력된 정규화되지 않은 원시 점수인 logits이다.

이 값을 확률로 변환하려면 SoftMax 계층을 통과해야 한다.

모든 &#129303;Transformers 모델은 이 logits 값을 출력하는데 그 이유는 일반적으로 학습을 위한 손실 함수는 최종 활성화 함수(activation function, e.g., SoftMax)와 실제 손실 함수(loss function, e.g., cross entropy)를 모두 사용하여 구현되기 때문이다.

In [9]:
wrapper.predictions = torch.nn.functional.softmax(wrapper.outputs.logits, dim = -1)
print(wrapper.predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


결과값은 [[0.0402, 0.9598], [0.9995, 0.0005]]와 같으며 사용자가 이해할 수 있는 확률 점수이다.

각 위치에 해당하는 레이블을 가져오기 위해 model.config의 `id2label` 속성값을 확인한다.

In [10]:
wrapper.model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [12]:
for prediction in wrapper.predictions:
    for label_id, label in wrapper.model.config.id2label.items():
        print(f"{label}: {prediction[label_id]:.4f}")

NEGATIVE: 0.0402
POSITIVE: 0.9598
NEGATIVE: 0.9995
POSITIVE: 0.0005


## Pipeline과 내부에서 실행되는 3단계 비교하기

In [16]:
from transformers import pipeline

wrapper.pipeline = pipeline("sentiment-analysis", wrapper.checkpoint)
print(wrapper.pipeline(wrapper.raw_inputs))

wrapper.init_wrapper()

[{'label': 'POSITIVE', 'score': 0.9598050713539124}, {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.87GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


# 2. Models

이 섹션에서는 모델을 생성하고 사용하는 방법을 자세히 살펴본다.

지정된 checkpoint를 바탕으로 모델을 인스턴스화할 때 편리한 AutoModel 클래스를 사용

`AutoModel`: checkpoint에 적합한 모델 아키텍처를 자동으로 추측한 다음 이 아키텍처 모델로 인스턴스화할 수 있는 영리한 wrapper class

그러나 사용하려는 모델의 유형을 알고있다면 해당 아키텍처를 직접 정의하는 클래스를 사용할 수 있다.

## Transformer Model 생성하기

BERT 모델과 함께 어떻게 작동하는지 살펴본다.

In [18]:
from transformers import BertConfig, BertModel

wrapper.config = BertConfig()

wrapper.model = BertModel(wrapper.config) # 또는 BertModel.from_pretrained(...)

print(wrapper.config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}


## Saving Method

`save_pretrained()`

In [19]:
wrapper.model.save_pretrained("../models/02.models")

- config.json: model architecture와 다양한 config 속성들, 모델 아키텍처 파악용
- pytorch_model.bin: state dictionary, 모델의 모든 가중치가 저장되어있음, 모델의 파라미터

## Transformer model을 활용한 추론(inference)

Transformer 모델은 토크나이저가 생성하는 숫자만 처리할 수 있다.

모델이 허용하는 입력은 행렬 형태의 2중 리스트만 허용한다.

In [22]:
wrapper.encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102]
]

wrapper.model_inputs = torch.tensor(wrapper.encoded_sequences)

print(wrapper.model_inputs)

tensor([[ 101, 7592,  999,  102],
        [ 101, 4658, 1012,  102],
        [ 101, 3835,  999,  102]])


## Model의 입력으로 텐서 활용

In [23]:
wrapper.output = wrapper.model(wrapper.model_inputs)
print(wrapper.output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1437, -1.8536, -0.7298,  ..., -0.7805,  0.3259,  0.4658],
         [ 0.3714, -0.9110, -0.8022,  ...,  0.3886,  0.8540,  0.6429],
         [-0.4563, -0.4854, -0.9873,  ...,  0.1407,  0.5035,  0.1391],
         [ 0.2318, -0.1520, -0.9008,  ...,  0.3321,  0.7286,  0.4390]],

        [[-1.9583, -1.9658, -0.4219,  ..., -0.8799,  0.8664,  0.7912],
         [-0.5089,  0.4376,  0.2652,  ..., -0.2958,  0.8589,  0.5303],
         [-0.3855, -0.0138, -0.4196,  ...,  0.0687,  0.5166, -0.3751],
         [-1.2665, -0.6342, -0.3569,  ...,  0.0830, -0.2257,  0.6619]],

        [[-0.4953, -1.8715, -0.8113,  ..., -0.5110,  1.4664,  1.1589],
         [ 0.9335, -0.7509, -0.5613,  ...,  0.4530,  1.5268, -0.5890],
         [-0.4374, -0.6369, -1.3684,  ..., -0.7098,  0.9980, -0.0311],
         [-0.1259,  0.3466, -0.7542,  ..., -0.7339,  0.2409,  1.0125]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.5303,  0.

# 3. Tokenizer

NLP의 핵심 구형 요소 중 하나. vocabulary에 없는 단어를 표현하기 위해 Unknown token [UNK]를 사용하는데 이를 줄이는 한 가지 방법은 문자기반 토크나이저를 사용하는 것이다.

## Character-based Tokenization

장점
- vocabulary의 크기가 매우 작음
- OOV(Out Of Vocabulary) 토큰이 훨씬 적음

단점
- 토큰 자체가 직관적이지 않음
- 토큰의 양이 너무 많음

단어기반 + 문자기반 => 하위단어 토큰화가 탄생함

## Subword Tokenization

빈번하게 사용하는 단어는 더 작은 하위단어로 분할하지 않고 희귀 단어를 의미있는 하위 단어로 분할한다는 원칙에 기반한다.

subwords를 연결하여 길이가 긴 복잡한 단어를 임의로 만들 수 있는 터키어와 같은 교착 언어에서 특히 유용함. ~~한국어도 교착어인디..?~~

#### 세부 기법들
- BBPE(Byte-level Byte Pair Encoding): GPT-2에 사용됨
- WordPiece: BERT에 사용됨
- SentencePiece, Unigram: 몇몇 다국어 모델에 사용됨

## Tokenizer 로딩 및 저장

`save_pretrained()`, `from_pretrained()`
: Model과 같은 방법으로 로딩 및 저장할 수 있다.

In [28]:
from transformers import AutoTokenizer, BertTokenizer

wrapper.auto_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
wrapper.bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

def is_same(sentenceA, sentenceB) -> bool:
    items_a = sentenceA.items()
    items_b = sentenceB.items()
    
    sorted(items_a, key = lambda x: x[0])
    sorted(items_b, key = lambda x: x[0])
    
    for item_a, item_b in zip(items_a, items_b):
        if item_a[0] != item_b[0]:
            return False
        if len(item_a[1]) != len(item_b[1]):
            return False
        for item_a1, item_b1 in zip(item_a[1], item_b[1]):
            if item_a1 != item_b1:
                return False
    return True

wrapper.inputs = "Using a Transformer network is simple"
print("Same") if is_same(wrapper.auto_tokenizer(wrapper.inputs), wrapper.bert_tokenizer(wrapper.inputs)) else print("Different")

wrapper.auto_tokenizer.save_pretrained("../models/02.models")

del is_same
wrapper.init_wrapper()


Same

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.81GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


## Encoding

Translating Text to Numbers Process
1. Tokenize
2. From Tokens To Input IDs

#### Tokenize

토큰화 프로세스는 Tokenizer의 `tokenize()`에 이해 수행됨.

In [29]:
wrapper.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

wrapper.sequence = "Using a Transformer network is simple"
wrapper.tokens = wrapper.tokenizer.tokenize(wrapper.sequence)

print(wrapper.tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


#### From Tokens To Input IDs

`convert_tokens_to_ids()` 메서드에 의해 처리

In [30]:
wrapper.ids = wrapper.tokenizer.convert_tokens_to_ids(wrapper.tokens)
print(wrapper.ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


## Decoding

Encoding의 반대 방향으로 진행되지만 `decode()`로 한 번에 해결

In [32]:
wrapper.decoded_string = wrapper.tokenizer.decode(wrapper.ids)
print(wrapper.decoded_string)

wrapper.init_wrapper()

Using a Transformer network is simple

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.80GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


# 4. 다중 시퀀스 처리

## model은 입력의 batch 형태를 요구한다.

model의 `_call_impl(self, *inputs, **kwargs)` 메서드를 보면 input string이 리스트 형태로 `*inputs`에 들어가있다.

input string이 한 개의 문자열 텐서(단일 리스트)라면 에러가 발생한다.

Tokenizer의 `__call__(...)` return 값이 2중 리스트인 것도 그런 이유 때문

따라서 단일 문자열을 텐서로 변환한 후 model에 전달하려고 할 때 텐서는 이렇게 만들어야 한다.
```python
import torch

ids = [] # tokens
input_ids = torch.tensor([ids])
```

## Padding / Truncation

문자열을 batch로 전달할 때 모든 문자열이 길이가 같을 수는 없다.

따라서 짧은 문자열에 padding을 추가해 attention_mask를 추가하거나 긴 문자열은 truncate하여 정사각형 텐서를 만들어야 한다.

방법은 Tokenizer 적용 시 두 개의 parameter를 추가한다.
- padding = True
- truncation = True